In [1]:
import pandas as pd
import numpy as np
import os

from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

In [2]:
source = 'source/UNPD_WCU2015_Unmet_need_Country Data Survey-Based.xlsx'

In [17]:
data = pd.read_excel(source, na_values='..', skiprows=3)

In [18]:
data = data.iloc[:, :8]

In [19]:
data.head()

,Country,ISO Code,Year(s),Age,Unmet need,Unnamed: 5,Unnamed: 6,Source
0,NaN,NaN,NaN,NaN,Total,Spacing,Limiting,NaN
1,Albania,8.0,2002,15-44,1.3,1,0.3,RHS
2,Albania,8.0,2008-2009,15-49,12.9,3.5,9.4,DHS
3,Algeria,12.0,2012,15-49,10.4,5.6,4.8,MICS
4,Armenia,51.0,2000,15-49,18.1,3.5,14.6,DHS


In [47]:
isnull = pd.isnull(data.ix[0])

for i in range(len(data.ix[0])):
    if not isnull.iloc[i]:
        print(data.columns[i])
        print('Unmet need ' + data.iloc[0, i])
        data = data.rename(columns={data.columns[i]: ('Unmet need ' + data.iloc[0, i])})

Unmet need
Unmet need Total
Unnamed: 5
Unmet need Spacing
Unnamed: 6
Unmet need Limiting


In [48]:
data.head()

,Country,ISO Code,Year(s),Age,Unmet need Total,Unmet need Spacing,Unmet need Limiting,Source
0,NaN,NaN,NaN,NaN,Total,Spacing,Limiting,NaN
1,Albania,8.0,2002,15-44,1.3,1,0.3,RHS
2,Albania,8.0,2008-2009,15-49,12.9,3.5,9.4,DHS
3,Algeria,12.0,2012,15-49,10.4,5.6,4.8,MICS
4,Armenia,51.0,2000,15-49,18.1,3.5,14.6,DHS


In [49]:
data = data.ix[1:]

In [50]:
data.head()

,Country,ISO Code,Year(s),Age,Unmet need Total,Unmet need Spacing,Unmet need Limiting,Source
1,Albania,8.0,2002,15-44,1.3,1,0.3,RHS
2,Albania,8.0,2008-2009,15-49,12.9,3.5,9.4,DHS
3,Algeria,12.0,2012,15-49,10.4,5.6,4.8,MICS
4,Armenia,51.0,2000,15-49,18.1,3.5,14.6,DHS
5,Armenia,51.0,2005,15-49,19.3,3.9,15.5,DHS


In [58]:
data.Age.unique()

array(['15-44', '15-49', '21-39', '18-40', '18-44', '20-49', '18-41',
       '18-49', '15-45'], dtype=object)

In [ ]:
# ages

In [64]:
age = data[['Age']].copy()

In [65]:
age['age'] = age['Age'].map(to_concept_id)

In [67]:
age.columns = ['name', 'age']

In [69]:
age = age.drop_duplicates()

In [70]:
age.to_csv('../ddf--entities--age.csv', index=False)

In [71]:
# country

In [79]:
country = data[['Country', 'ISO Code']]

In [80]:
country = country.drop_duplicates().copy()

In [81]:
country.columns = ['name', 'iso_code']

In [82]:
country.iso_code = country.iso_code.map(int)

In [124]:
country.to_csv('../ddf--entities--iso_code.csv', index=False)

In [85]:
# unmet need

In [89]:
unmet_need = pd.DataFrame([['limiting', 'Unmet need Limiting'],
                           ['spacing', 'Unmet need Spacing']
                          ], columns=['unmet_need_section', 'name'])

In [90]:
unmet_need.to_csv('../ddf--entities--unmet_need_section.csv', index=False)

In [88]:
# concepts

In [92]:
cdf = pd.DataFrame([['name', 'Name', 'string'],
                    ['iso_code', 'ISO Code', 'entity_domain'],
                    ['year', 'Year(s)', 'time'],
                    ['age', 'Age', 'entity_domain'],
                    ['unmet_need_section', 'Unmet need sections', 'entity_domain'],
                    ['unmet_need', 'Unmet need', 'measure']
                   ], columns=['concept', 'name', 'concept_type'])

In [93]:
cdf.to_csv('../ddf--conceptps.csv', index=False)

In [94]:
# datapoints

In [99]:
dps = data[['ISO Code', 'Year(s)', 'Age', 'Unmet need Total',
            'Unmet need Spacing', 'Unmet need Limiting']].copy()

In [100]:
dps.columns = ['iso_code', 'year', 'age', 'total', 'spacing', 'limiting']

In [101]:
dps.iso_code = dps.iso_code.map(int)
dps.age = dps.age.map(to_concept_id)

In [102]:
dps = dps.set_index(['iso_code', 'year', 'age'])

In [106]:
total = dps.total.copy()

In [108]:
total.name = 'unmet_need'

In [110]:
total.dropna().reset_index().to_csv('../ddf--datapoints--unmet_need--by--iso_code--age--year.csv', index=False)

In [111]:
sections = dps[['spacing', 'limiting']].copy()

In [116]:
sections = sections.stack().reset_index()

In [119]:
sections.columns = ['iso_code', 'year', 'age', 'unmet_need_section', 'unmet_need']

In [122]:
sections.to_csv('../ddf--datapoints--unmet_need--by--iso_code--age--unmet_need_section--year.csv', index=False)

In [125]:
create_index_file('../')

,key,value,file
0,concept,name,ddf--conceptps.csv
1,concept,concept_type,ddf--conceptps.csv
0,"iso_code,age,unmet_need_section,year",unmet_need,ddf--datapoints--unmet_need--by--iso_code--age...
0,"iso_code,age,year",unmet_need,ddf--datapoints--unmet_need--by--iso_code--age...
0,age,name,ddf--entities--age.csv
0,iso_code,name,ddf--entities--iso_code.csv
0,unmet_need_section,name,ddf--entities--unmet_need_section.csv
